# Data Mining Process

En este notebook realizaremos todo el Data Mining Process. En el apartado 2. se pueden ver los pasos a realizar:


1. Entender el problema (Problem Understanding)
2. Data Mining Process
    * Obtener los datos (Data collect)
    * Procesando los datos (Data Processing)
    * Exploración estadística de datos (Statistical Data Exploration)
    * Modelado del análisis (Analysis Modeling)
3. Resultados (Results)



Antes de nada, importamos las librerias y funciones necesarias:

In [433]:
import tensorflow as tf
import pandas as pd
import skmob
from skmob.measures.individual import home_location
from skmob.io.file import load_geolife_trajectories


## 2.1. Obtener los datos

Para cargar los datos, usamos la función 'load_geolife_trajectories' que nos proporciona skmob. Esta función se encarga de cargar los datos de GeoLife Trajectories, solo hay que indicarle el path del directorio Data, y los usuarios que queremos cargar. Es opcional pasarle algunos parámetros de filtrado y compresión.

Esta función, por cómo esta implementada, solo contempla la sintaxis de las rutas de archivos en Linux, por lo que en windows no funcionará.

In [434]:
path = "/home/alonso/Documentos/tfg/Geolife_Trajectories/"
# Cargamos los datos de los usuarios 113
traj_df = load_geolife_trajectories(path, user_ids=[120,121,122,123],
                                    compress_kwargs={'spatial_radius_km': 0.0})

## 2.2. Procesando los datos

En esta segunda fase, es importante tener una "visión global" de los datos, con el fin de entenderlos mejor. Una buena idea es ver si hay datos nulos en la información que tenemos, ya que estos deberían ser tratados. Se puede ver que en esta ocasión, no los hay. Si los hubiese, una opción podría ser la de descartar estos datos. Para realizar esto último, se podría hacer uso del método dropna() para descartar las filas con algún dato nulo los datos. Otra opcion sería remplazar los nulos por algun otro valor con fillna()

In [435]:
df = pd.DataFrame(traj_df)
print('Total de valores nulos de cada columna: ')
print(df.isna().sum() )
print('\nVistazo de los datos:')
df.head()


Total de valores nulos de cada columna: 
lat         0
lng         0
datetime    0
uid         0
dtype: int64

Vistazo de los datos:


,lat,lng,datetime,uid
0,39.982375,116.320442,2009-09-19 07:11:37,120
1,39.982380,116.320448,2009-09-19 07:11:48,120
2,39.982380,116.320455,2009-09-19 07:11:50,120
3,39.982397,116.320455,2009-09-19 07:11:52,120
4,39.982425,116.320460,2009-09-19 07:11:54,120


Para obtener la localización de las casas de cada individuo en el dataset, scikit-mobility nos proporciona la función home_location(). Esta función observa la localización más frecuentada en la noche (Entre las 22:00 y las 07:00) por los usuarios y designa a este como su casa. Estas horas son modificables, aunque no lo haremos ya que lo normal, es estar en casa entre esas horas.

In [436]:
home_loc_df = home_location(df)
home_loc_df.head()

100%|██████████| 4/4 [00:00<00:00, 141.83it/s]


,uid,lat,lng
0,120,42.405338,117.249225
1,121,39.906183,116.379599
2,122,39.968092,116.399647
3,123,34.264762,108.939026


Como ya tenemos los dos dataframes, uno con los datos GPS y otro con las localizaciones de las casas de cada individuo, es necesario trabajar con ellos para obtener los datos que buscamos.

Añadimos al dataframe de los datos GPS (nuestro 'df') una columna extra que indicará si esta en casa y la inicializamos con 0. Al dataframe con la localizacion de las casas de los individuos (home_loc_df) le añadimos una columna extra de mismo nombre, con valor 1, ya que en esa posición se encuentra la casa.

Despues, hacemos lo que en base de datos se conoce como un JOIN LEFT usando como key los valores 'lat', 'lng' y 'uid'. Con conseguiremos tener la una columna at_home_y, con el valor de 1 cuando se encuentre una coincidencia y un valor nulo cuando no y otra at_home_x con todos sus valores 0. 

Por último ya solo quedaría crear y asignar a una nueva columna at_home el valor de la columna at_home_y y rellenar los valores nulos con los de la columna at_home_x (que tienen valor 0). Despues ya solo quedaría descartar las columnas que sobran y quedarmos con at_home

In [441]:
df['at_home'] = 0
home_loc_df['at_home'] = 1
df = pd.merge(df, home_loc_df, on=['lat','lng','uid'], how='left')
df['at_home'] = df['at_home_y'].fillna(df['at_home_x'])  
df = df.drop(['at_home_x','at_home_y'], axis=1)

11


,lat,lng,datetime,uid,at_home
0,39.982375,116.320442,2009-09-19 07:11:37,120,0.0
1,39.982380,116.320448,2009-09-19 07:11:48,120,0.0
2,39.982380,116.320455,2009-09-19 07:11:50,120,0.0
3,39.982397,116.320455,2009-09-19 07:11:52,120,0.0
4,39.982425,116.320460,2009-09-19 07:11:54,120,0.0


## 2.3. Exploración estadística de datos

In [448]:
df.describe()



,lat,lng,at_home
count,92830.000000,92830.000000,92830.000000
mean,38.559663,115.764088,0.000118
std,4.675042,2.255154,0.010885
min,22.534653,108.726840,0.000000
25%,39.964671,116.231612,0.000000
50%,39.976559,116.398177,0.000000
75%,40.131145,116.474284,0.000000
max,42.551380,117.957961,1.000000


lat         92819
lng         92819
datetime    92819
uid         92819
at_home     92819
dtype: int64

In [400]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [388]:
scaler = StandardScaler()

df_scaled = scaler.transform(df)


NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.